# EEG - Flow

## 1. Convert XDF to FIFF

The Lab Streaming Layer streams (LSL) stored in the *.xdf* file must be loaded and converted into a FIFF format.
The *.xdf* file contains:
- *eegoSports 000xxx*: the EEG data stream, with `000xxx` the serial number (S/N) of the amplifier.
- *Oddball_task*: triggers of the oddball task, duplicate of the hardware triggers from received on the `TRIGGER` channel of the EEG data stream.
- *OBS_webcam* (task-UT-only): Frame IDs from the webcam OBS video stream.
- *OBS_gameplay* (task-UT-only): Frame IDs from the screen capture OBS video stream.
- *UT_GameEvents* (task-UT-only): Events from Unreal Tournament.
- *MouseButtons* (task-UT-only): Mouse clicks and button events.
- *MousePosition* (task-UT-only): Mouse X/Y position.
- *Keyboard* (task-UT-only): Keyboard button events.

TO DO: run all 4 files at the same time in parallel? need to test on unige wired network. VPN is being way too slow

Last edit: 05.05.2023 00:20
@anguyen

In [ ]:
import itertools
import multiprocessing as mp
import tqdm

from eeg_flow.tasks import convert_xdf_to_fiff

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
"""
participants_with_group: write all participants in a list
tasks                  : "oddball" and/or "UT" in a list
runs                   : 1 and/or 2
"""

participants_with_group = ["P25-G1"]
tasks = ["oddball"]
runs = [1, 2]

########
iterations = list(itertools.product(participants_with_group, tasks, runs))
iterations[:] = [(t[0].split('-') + list(t[1:])) for t in iterations]
iterations

In [ ]:
# attempt at a progress bar but this only start showing it once it's done...
def run_pbar():
    with mp.Pool(processes=8, maxtasksperchild=20) as p:
        with tqdm.tqdm(total=len(iterations)) as pbar:
            for _ in p.starmap(convert_xdf_to_fiff, iterations):
                pbar.update()

In [ ]:
def run():
    with mp.Pool(processes=8, maxtasksperchild=20) as p:
        p.starmap(
            convert_xdf_to_fiff, iterations
        )

In [ ]:
%%time
run()

this ends up savings a fif output